In [2]:
#import Control
class EntryVehicle(object):
    
    '''
    Defines an EntryVehicle class:
    members:
        area - the effective area, m^2
        CD   - a multiplicative offset for the vehicle's drag coefficient
        CL   - a multiplicative offset for the vehicle's lift coefficient
        g0 - sea level Earth gravity, used in mass rate of change, m/s^2
    methods:
        aerodynamic_coefficients(Mach, alpha) - computes the values of CD and CL for the current Mach values, angle of attack
        BC(mass, Mach) - computes the vehicle's ballistic coefficient as a function of its mass. Drag coefficient is calculated by default at Mach 24.
    '''

    def __init__(self, area=0.4839, mass=907.2):
        # need to put real value for cav thrust, ISP
        self.area = area
        self.mass = mass
        self.g0 = 9.81

    def aerodynamic_coefficients(self, M, alpha):
        self.M = self.Control.mach()
        self.alpha = self.Control.alpha()
        from math import exp
        """ Returns aero coefficients CD and CL. Supports ndarray Mach numbers. """
        [cl0, cl1, cl2, cl3] = [-0.2317, 0.0513, 0.2945, -0.1028]
        [cd0, cd1, cd2, cd3] = [0.024, 7.24*exp(-4), 0.406, -0.323]
        
        cL = cl1*self.alpha + cl2*exp(cl3*self.M) + cl0
        cD = cd1*self.alpha**2 + cd2*exp(cd3*self.M) + cd0
        LoD = cL/cD
        return cD, cL
    
    def ballistic_coefficients(self, mass=907.2, area=0.4839):
        self.mass = mass
        self.cD = self.aerodynamic_coefficients(M, alpha)[0]
        self.area = area
        
        bc = mass/(self.cD*area)
        return bc

In [3]:
import isacalc as isa
atmos = isa.get_atmosphere

class Planet:
    def __init__(self, name='Earth', rho0=0, scaleHeight=0, model='exp', da=False):

        self.name = name.capitalize()
        self._da = da  # Differential algebraic inputs

        if self.name == 'Earth':
            self.radius = 6378.1e3
            self.omega = 7.292115e-5
            self.mu = 3.98600e14
            
            self.atmosphere= self.__atmos_earth

        elif self.name == 'Mars':
            self.radius = 3396.2e3
            self.omega = 7.095e-5
            self.mu = 4.2830e13

            if model is 'exp':
                self.rho0 = (1+rho0)*0.0158
                self.scaleHeight = (1+scaleHeight)*9354.5
                self.atmosphere = self.__exp_model_mars

            else:
                # Sample MG and store interpolators for density and speed of sound
                self.atmosphere = self.__MG_model_mars
        else:
            print('Input planet name, '+ self.name +', is not valid')
    def __atmos_earth(self, h):
        atmosphere = isa.get_atmosphere
        T, P, rho, a, mu = isa.calculate_at_h(h, atmosphere)
        return rho, a
        
    def __exp_model_mars(self, h):
        ''' Defines an exponential model of the atmospheric density and local speed of sound as a function of altitude. '''
        if self._da:
            from pyaudi import exp
            scalar=False
            try:
                h[0]
            except:
                scalar=True
                h = [h]
            #Density computation:
            rho = [self.rho0*exp(-hi/self.scaleHeight) for hi in h]

            # Local speed of sound computation:
            coeff = [223.8, -0.2004e-3, -1.588e-8, 1.404e-13]
            a = [sum([c*hi**i for i,c in enumerate(coeff)]) for hi in h]
            if scalar:
                a = a[0]
                rho = rho[0]

        else:
            from numpy import exp
            #Density computation:
            rho = self.rho0*exp(-h/self.scaleHeight)

            # Local speed of sound computation:
            coeff = [223.8, -0.2004e-3, -1.588e-8, 1.404e-13]
            a = sum([c*h**i for i,c in enumerate(coeff)])

        return rho,a

    def __MG_model_mars(self, h):
        ''' Interpolates data from an MG profile '''
        return self.density(h),self.speed_of_sound(h)

    def updateMG(date=[10,29,2018], latitude=0, longitude=0, dustTau=0, rpscale=0):
        ''' Calls MG '''
        return

<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\User\AppData\Local\Temp/ipykernel_14928/1202008157.py:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if model is 'exp':


In [4]:
import numpy as np
from numpy import sin, cos, tan
#from Entry_Vehicle import EntryVehicle
#import Planet

class Entry(object):
    """  Basic equations of motion for unpowered and powered flight through an atmosphere. """

    def __init__(self, PlanetModel=Planet('Earth'), VehicleModel=EntryVehicle(), Coriolis=False, Powered=False, Energy=False, Altitude=False, DifferentialAlgebra=False, Scale=False, Longitudinal=False, Velocity=False):


        
        self.planet = PlanetModel
        self.vehicle = VehicleModel
        self.powered = Powered
        self.drag_ratio = 1
        self.lift_ratio = 1
        self.nx = 6  # [r,lon,lat,v,gamma,psi]
        self.nu = 2  # bank command, angle of attack
        self.__jacobian = None  # If the jacobian method is called, the Jacobian object is stored to prevent recreating it each time. It is not constructed by default.
        self.__jacobianb = None
        self._da = DifferentialAlgebra
        self.planet._da = DifferentialAlgebra

        #todo: Non-dimensionalizing the states
        if Scale:
            self.dist_scale = self.planet.radius
            self.acc_scale = self.planet.mu/(self.dist_scale**2)
            self.time_scale = np.sqrt(self.dist_scale/self.acc_scale)
            self.vel_scale = np.sqrt(self.dist_scale*self.acc_scale)
            self.long_scale = self.theta_us/pi
            self.lat_scale = self.phi_us/pi
            self.fpa_scale = self.gamma_us/pi
            self.ha_scale = self.psi_us/pi
            self.mass_scale = 1

        else:  # No scaling
            self.dist_scale = 1
            self.acc_scale = 1
            self.time_scale = 1
            self.vel_scale = 1
        

        self.dyn_model = self.__entry_3dof

    def update_ratios(self, LR, DR):
        self.drag_ratio = DR
        self.lift_ratio = LR

    def DA(self, bool=None):
        if bool is None:
            return self._da
        else:
            self._da = bool
            self.planet._da = bool

# Dynamics model
        
    def __entry_3dof(self, s, alpha, sigma, t):
        r, theta, phi, v, gamma, psi =  self.s
        g = self.gravity(r)
        omega = self.planet.omega
        h = (r - self.planet.radius)/self.dist_scale
        rho, a = self.planet.atmosphere(h*self.dist_scale)
        M = v*self.vel_scale/s_s
        alpha = self.Control.alpha
        sigma = self.Control.sigma
        cD, cL = self.vehicle.aerodynamic_coefficients(M, alpha)
        m = self.vehicle.mass
        f = np.squeeze(0.5*rho*self.vehicle.area*v**2/m)*self.dist_scale  # vel_scale**2/acc_scale = dist_scale 
        L = f*cL*self.lift_ratio
        D = f*cD*self.drag_ratio      
        
        dr = v*sin(gamma)
        dtheta = v*cos(gamma)*cos(psi)/r*cos(phi)
        dphi = v*cos(gamma)*sin(psi)/r
        dv = -D/m - g*sin(gamma) + omega**2*r*cos(phi)*(sin(gamma)*cos(phi) - cos(gamma)*sin(phi)*sin(psi))
        dgamma = L*cos(sigma)/m*v + cos(gamma)*((v**2-g*r)/r*v) + 2*omega*sin(psi)*cos(phi) + omega**2*r*cos(phi)*(cos(phi)*cos(gamma)+sin(phi)*sin(psi)*sin(gamma)) 
        dpsi = L*sin(sigma)/m*v*cos(gamma) - v*cos(gamma)*cos(psi)*tan(phi)/r + 2*omega*(tan(gamma)*cos(phi)*sin(psi)-sin(phi)) - omega**2*r*sin(phi)*cos(phi)*cos(psi)/cos(gamma)

        return np.array([dr, dtheta, dphi, dv, dgamma, dpsi])
    
    def aeroforces(self, r, v, m):
        """  Returns the aerodynamic forces acting on the vehicle at a given radius, velocity and mass. """

        h = r - self.planet.radius
        rho, a = self.planet.atmosphere(h)
        M = v/a
        # cD, cL = self.vehicle.aerodynamic_coefficients(M)
        cD, cL = self.vehicle.aerodynamic_coefficients(v)
        f = 0.5*rho*self.vehicle.area*v**2/m
        L = f*cL*self.lift_ratio
        D = f*cD*self.drag_ratio
        return L, D

    def gravity(self, r):
        """ Returns gravitational acceleration at a given planet radius based on quadratic model 
        
            For radius in meters, returns m/s**2
            For non-dimensional radius, returns non-dimensional gravity 
        """
        return self.planet.mu/(r*self.dist_scale)**2/self.acc_scale

In [5]:
import sys
sys.path.append('C:\\Users\\User')
from scipy import cos
import pybrain
from pybrain.rl.environments.episodic import EpisodicTask
#import Entry_Vehicle
#import Entry
#import Planet

import random
import math


class Control(EpisodicTask):

    # number of steps of the current trial
    steps = 0

    # number of the current episode
    episode = 0
    
    maxSteps = 99
    
    resetOnSuccess = True

    def __init__(self):
        self.reset()
        self.cumreward = 0

    def reset(self):
        self.state = self.GetInitialState()
        alpha = random.uniform(-20, 20)
        sigma = 0
    
    def getObservation(self):    
        self.state = self.Entry.__entry_3dof(self, s, alpha, sigma, t)
        return [(self.state)]
        
    def performAction(self, action):
        if self.done > 0:
            self.done += 1            
        else:
            self.state = self.DoAction(action, self.state)
            self.r, self.done = self.GetReward(self.state, action)
            self.cumreward += self.r
            
    def getReward(self):
        return self.r    

    def GetInitialState(self):
        # Initialize state values
        self.StartEpisode()
        r = 6448
        theta = 0
        phi = 0
        v = 6500
        gamma = 0
        psi = 90
        
        return [r, theta, phi, v, gamma, psi]
  
    def FinalState(self):
        # set final target values
        r = 6408
        theta = 50
        phi = 0
        v = 1600
        gamma = 0
        psi = 90
        
        return [r, theta, phi, v, gamma, psi]
    
    def StartEpisode(self):
        self.steps = 0
        self.episode = self.episode + 1
        self.done = 0
        
    def isFinished(self):
        if self.done>=1 and self.resetOnSuccess:
            self.reset()
            return True
        else:
            return self.done>=3
    

    def GetReward(self, s, a):
        #todo: formulate reward function
        # satisfy constraints either here or in doaction
        r = 0
        f = 0
        
        if (self.s-self.FinalState) == np.zeros(6):
            r = 100
        elif np.absolute(self.s-self.FinalState) < np.array([10, 5, 5, 100, 5, 5]):
            r = 50
        elif np.absolute(self.s-self.FinalState) < np.array([20, 10, 10, 400, 10, 10]):
            r = 10
        elif np.absolute(self.s-self.FinalState) < np.array([30, 20, 20, 1000, 20, 20]):
            r= 2
        elif np.absolute(self.s-self.FinalState) < np.array([40, 30, 30, 2000, 30, 30]):
            r = 0 
        elif np.absolute(self.s-self.FinalState) > np.array([40, 30, 30, 2000, 30, 30]):
            r = -1
        elif np.absolute(self.s-self.FinalState) > np.array([45, 30, 30, 2500, 30, 30]):
            r = -5
        elif np.absolute(self.s-self.FinalState) > np.array([50, 45, 45, 3000, 45, 45]):
            r = -10
        if self.steps >= self.maxSteps:
            f = 5
   
        return r, f

    def DoAction(self, a, s):

        alpha = self.alpha
        sigma = self.sigma
        # todo: satisfy constraints
        self.steps = self.steps + 1
        s_new = s + self.Entry.__entry_3dof(s, alpha, sigma, t)

        
        return [s_new]
    
    def mach(self):
        M = self.Entry.__entry_3dof.M
        
        return M
    
    def alpha(self):
        #alpha = self.get_action[0]
        ''''
        if self.step=0:
            alpha = random.uniform(-20, 20)
        else:
            pass
            '''
        alpha_old = self.AlphaOld
        alpha_new = random.uniform(-20, 20)
        dalpha = alpha_new - alpha_old
        
        if dalpha<-5:
            alpha = alpha_old - 5
        elif dalpha>5:
            alpha = alpha_old + 5
        else:
            alpha = alpha_new
        return alpha
    
    def AlphaOld(self):
        alpha_old = self.alpha
        return alpha_old
    
    def sigma(self):
        #sigma = self.get_action[1]
        '''  
        if self.step=0:
            sigma = 0
        else:
            pass
        '''
        sigma_old = self.SigmaOld
        sigma_new = random.uniform(-90, 90)
        dsigma = sigma_new - sigma_old
        
        if dsigma<-15:
            sigma = sigma_old - 15
        elif dsigma>15:
            sigma = sigma_old + 15
        else:
            sigma = sigma_new
        return sigma
    
    def SigmaOld(self):
        sigma_old = self.sigma
        return sigma_old
    
    def contraints(self):
        V = self.Entry.__entry_3dof[3]
        K = 9.289e-9
        rho = self.Planet.atmosphere[0]
        Cd, Cl = self.EntryVehicle.aerodynamic_coefficients(M, alpha)
        D = 0.5 * rho * V **2 * S * Cd
        L = (Cl/Cd) * D
        #aerodynamic heating
        Qd = K * rho ** 0.5 * V ** 3
        #dynamic pressure
        P_d = 0.5 * rho * V ** 2
        #normal load
        n_L = math.sqrt(L**2+D**2)/(self.mass*self.gravity)
        return [[Qd, P_d, n_L]]

In [6]:
class HrvEnv(Env):
    print_interval = 1
    def __init__(self, logger = None):
        self.env = Hrv()
        self.noiseRange = 1.0
        self.noiseMax = 1.0
        self.om = 0
        self.a = 0.6
        self.b = 0.4
        self.t = 0
        self.totStep = 0
        self.r = 0
        self.ep = 0
        if logger==None:
            self.perfs = result_log(algo="DDPG", l1=20, l2=10)
        else:
            self.perfs = logger
        self.actif = True
        #self.plot = result_plot()
    
    def state(self):
        return [self.env.getObservation()]
    def act(self, action):
        actNoise = action + self.noise_func()
        self.env.performAction(actNoise[0])
        r = self.env.getReward()
        self.t += 1
        self.r += r
        return actNoise, [r]
    def reset(self, noise=True):
        self.actif = True
        self.env.reset()
        self.om = 0
        self.totStep+=self.t
        if self.totStep != 0:
            self.perfs.addData(self.totStep, self.t, self.r)
        self.t = 0
        self.r = 0
        self.ep += 1
        if not noise:
            self.noiseRange = 0.0
        else:
            self.noiseRange = random.uniform(0.,self.noiseMax)
    def noise_func(self):
        self.om = self.om-self.a*self.om + self.b*random.gauss(0,1)*self.noiseRange
        return self.om
    def isFinished(self):
        if self.actif and not self.env.isFinished():
            return False
        else:
            self.actif = False
            return True
    def getActionSize(self):
        return 2
    def getStateSize(self):
        return 6
    def getActionBounds(self):
        return np.array[[-20], [20],
               [-90], [90]]
    def printEpisode(self):
        print(time.strftime("[%H:%M:%S]"), " Episode : " , self.ep, " steps : ", self.t, " reward : ", self.r, "noise : ", self.noiseRange)
    def performances(self):
        pass#self.plot.clear()
        #self.plot.add_row(self.perfs)

NameError: name 'Env' is not defined

In [9]:
import random
import sys
sys.path.append('C:\\Users\\User')
import DDPG
from collections import deque
import time

from DDPG.core.networks.simple_actor_network import simple_actor_network
from DDPG.core.networks.simple_critic_network import simple_critic_network
from DDPG.core.helper.tensorflow_grad_inverter import grad_inverter
from DDPG.environement.env import Env



"""A définir"""
def void_noise_func(t):
    return 0.0
def void_env_act(action, store):
    return 0
def void_env_state():
    return [0]
def void_env_ini():
    pass
def void_env_draw():
    pass
def void_env_stop_signial():
    return False

class DDPG(object):
    """DDPG's main structure implementation"""
    
    def __init__(self, env, actor = None, critic = None):
        self.env = env
        if not isinstance(env,Env):
            print ("error")
        
        s_dim = env.getStateSize()
        a_dim = env.getActionSize()
        action_bounds = env.getActionBounds()
        
        if actor == None:
            self.actor = simple_actor_network(s_dim, a_dim)
        else:
            self.actor = actor
        if critic == None:
            self.critic = simple_critic_network(s_dim, a_dim, action_bounds)
        else:
            self.critic = critic 
        self.buffer = deque([])
        self.buffer_size = 100000
        self.buffer_minimum = 100
        self.minibatch_size = 64
        self.t = 0
        
        self.grad_inv = grad_inverter(action_bounds)
        
        self.train_loop_size = 1
        self.totStepTime = 0
        self.totTrainTime = 0
        self.batchMix = 0
        self.calcTrain = 0
        self.time0 = 0
        self.time1 = 0
        self.time2 = 0
        self.numStep = 0
        self.stepsTime = 0
    
    def react(self, state):
        act = self.actor.action_batch(state)
        return act
        
    def store_transition(self, s_t, a_t, r_t, s_t_nxt):
        for i in range(len(s_t)):
            if(len(self.buffer)>=self.buffer_size):
                self.buffer[random.randint(self.buffer_size/5, self.buffer_size-1)] = [list(s_t[i]), list(a_t[i]), r_t[i], list(s_t_nxt[i])]
            else:
                self.buffer.append([list(s_t[i]), list(a_t[i]), r_t[i], list(s_t_nxt[i])])
        
            
    def train_Minibatch(self):
        if(len(self.buffer)>self.buffer_minimum):
            rewards = []
            states = []
            nxt_states = []
            actions_batch = []
            batchMix = time.time()
            for i in range(self.minibatch_size):
                r= random.randint(0, len(self.buffer)-1)
                tmp = self.buffer[r]
                rewards.append([tmp[2]])
                states.append(tmp[0])
                nxt_states.append(tmp[3])
                actions_batch.append(tmp[1])
            self.batchMix+=time.time()-batchMix
            
            calcTrain = time.time()
            y = self.critic.y_val_calc(rewards, self.critic.q_val_batch(nxt_states, self.actor.action_batch(nxt_states, True), True))
            self.critic.batch(states, actions_batch, y)
            self.critic.update()
            self.time0 += time.time() - calcTrain
            
            time1 = time.time()
            actions = self.actor.action_batch(states)
            actionGradients = self.grad_inv.invert(self.critic.actionGradient_batch(states, actions), actions)
            self.actor.batch(states, actionGradients)
            self.actor.update()
            self.time1 += time.time() -time1
            
            time2 = time.time()
            self.critic.updateTarget()
            self.actor.updateTarget()
            self.time2+= time.time() -time2
            self.calcTrain+= time.time() - calcTrain
            
    def step(self, train):
        state = list(self.env.state())
        action = self.react(state)
        (action, reward) = self.env.act(action)
        if train:
            self.store_transition(state, action, reward, self.env.state())
        self.t += 1
        return reward
    
    def episode(self, max_t=float("inf"), train=True):
        while self.t<max_t and not self.env.isFinished():
            stepTime = time.time()
            self.step(train)
            if train :
                self.totStepTime += time.time() - stepTime
                for i in range(self.train_loop_size):
                    trainTime = time.time()
                    self.train_Minibatch()
                    self.totTrainTime += time.time() - trainTime
                self.numStep+=1
    def M_episodes(self, M, T=float("inf"), train=True):
        for i in range(M):
            self.t = 0
            if self.env.isFinished():
                self.env.reset()
            self.episode(T, train)
            if i % self.env.print_interval == 0:
                self.stepsTime += self.totStepTime + self.totTrainTime
                self.env.printEpisode()
                self.env.draw()
                #print "step time : ", self.totStepTime/(self.totStepTime+self.totTrainTime) , "train time : ", self.totTrainTime/(self.totStepTime+self.totTrainTime)
                #print "train decomp -> batch mix : ", self.batchMix/(self.batchMix+self.calcTrain), " calculations : ", self.calcTrain/(self.batchMix+self.calcTrain)
                #print "time 0 : ", self.time0/(self.time0 + self.time1+self.time2), "time 1 : ", self.time1/(self.time0 + self.time1+self.time2), "time 2 : ", self.time2/(self.time0 + self.time1+self.time2)                
                print ("Steps/minutes : " , 60.0 / (self.stepsTime / self.numStep))             
                self.totStepTime = 0
                self.totTrainTime = 0
                self.batchMix = 0
                self.calcTrain = 0
                self.time0 = 0
                self.time1 = 0
                self.time2 = 0
#            if i % 1 == 10:
#                self.env.reset(noise=False)
#                self.episode(T, train=False)
#                #self.stepsTime += self.totStepTime + self.totTrainTime
#                self.env.printEpisode()
#                self.env.draw()
#                #print "Steps/minutes : " , 60.0 / (self.stepsTime / self.numStep)                
#                self.totStepTime = 0
#                self.totTrainTime = 0
#                self.batchMix = 0
#                self.calcTrain = 0
#                self.time0 = 0
#                self.time1 = 0
#                self.time2 = 0
        if self.env.isFinished():
            self.env.reset()
    def buffer_flush(self):
        self.buffer = []

In [15]:
from DDPG.core.DDPG_core import DDPG
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf

import DDPG.environement.instance.mountainCarEnv as mc
from DDPG.core.networks.simple_actor_network import simple_actor_network
from DDPG.logger.result import result_log

l1 = 20
l2 = 10
rate = 0.001

env = mc.MountainCarEnv(result_log("DDPG", l1, l2))
a_c = DDPG(env, actor = simple_actor_network(2, 1, l1_size = l1, l2_size = l2, learning_rate = rate))

def voidFunc():
    pass

env.extern_draw = voidFunc

def doEp(M, T=float("inf")):
    a_c.M_episodes(M, T)
    env.perfs.save()
doEp(4000)

UnboundLocalError: local variable 'tf' referenced before assignment

In [18]:
from DDPG.core.DDPG_core import DDPG
import numpy as np

import DDPG.environement.instance.mountainCarEnv as mc
from DDPG.core.networks.simple_actor_network import simple_actor_network
from DDPG.core.networks.simple_critic_network import simple_critic_network

import matplotlib.pyplot as plt
from DDPG.logger.result import result_log

l1 = 20
l2 = 10

logger = result_log("DDPG", l1, l2, "simple_"+str(l1)+"_"+str(l2))

env = mc.MountainCarEnv(logger)
a_c = DDPG(env, actor = simple_actor_network(2, 1, l1_size = l1, l2_size = l2, learning_rate = 0.005), critic = simple_critic_network(2, 1, l1_size = 20, l2_size = 10, learning_rate = 0.01))
    
def draw_politic():
    plt.close()
    ac= a_c
    img = np.zeros((200, 200))
    pos = -1.
    batch = []
    for i in range(200):
        vel = -1.
        pos += 0.01
        for j in range(200):
            vel += 0.01
            batch.append([pos, vel])
    pol = ac.react(batch)
    b=0           
    print("politic max : ", max(pol), " politic min : ", min(pol))
    for i in range(200):
        for j in range(200):
            img[-j][i] = max(-1, min(1.0, pol[b]))
            b += 1
    img[0][0] = -1
    img[-1][-1] = 1.0
    plt.imshow(img, extent=(-1.0,1.0,-1.0,1.0))
    plt.show(block=False)
def draw_episode():
    act = a_c.actor
    env.reset()
    env.noiseRange=0.0
    while not env.isFinished():
        plt.scatter((env.state()[0][0]),(env.state()[0][1]), c="white")
        env.act(act.action_batch(env.state()))

def perfs():
    env.performances()

def voidFunc():
    pass

env.extern_draw = voidFunc
def draw_buffer():
    for i in range(len(a_c.buffer)):
        plt.scatter((a_c.buffer[i][0][0]+1.)*100, (a_c.buffer[i][0][1]+1.)*100)

def doEp(M, T=float("inf")):
    a_c.M_episodes(M, T)
    env.perfs.save()

UnboundLocalError: local variable 'tf' referenced before assignment